# Sentinel-2などへのアクセス方法について  
- [参考note](https://note.com/smartagri/n/n57ab1dd47da6#c068a32d-20cc-4ba1-9c03-e8b554d6440c)
- [公式ガイドgithub](https://github.com/eu-cdse/notebook-samples/blob/c0e0ade601973c5d4e4bf66a13c0b76ebb099805/sentinelhub/migration_from_scihub_guide.ipynb)

## 最終行の`from utils import plot_image`でエラーが出る時の対処法  

[参考](https://ja.stackoverflow.com/questions/49412/%E3%82%B3%E3%83%BC%E3%83%89%E3%82%92jupyter-notebook%E3%81%AE%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E3%81%AB%E4%BF%9D%E5%AD%98%E3%81%97%E3%81%A6import%E3%81%99%E3%82%8B%E3%81%A8%E3%82%A8%E3%83%A9%E3%83%BC%E3%81%AB%E3%81%AA%E3%82%8B)  
1. [github](https://github.com/eu-cdse/notebook-samples/blob/c0e0ade601973c5d4e4bf66a13c0b76ebb099805/sentinelhub/migration_from_scihub_guide.ipynb)
  のutils.pyを直接読み込む  
2. TextFileでutil.pyを作成し、コードをコピペする。  
   コードの中身 
   ```python  
   """
    Utilities used by example notebooks
    """

    from __future__ import annotations

    from typing import Any

    import matplotlib.pyplot as plt
    import numpy as np


    def plot_image(
    image: np.ndarray, factor: float = 1.0, clip_range: tuple[float, float] | None = None, **kwargs: Any
    ) -> None:
    """Utility function for plotting RGB images."""
    _, ax = plt.subplots(nrows=1, ncols=1, figsize=(15, 15))
    if clip_range is not None:
        ax.imshow(np.clip(image * factor, *clip_range), **kwargs)
    else:
        ax.imshow(image * factor, **kwargs)
    ax.set_xticks([])
    ax.set_yticks([])  
    
    ```

In [1]:
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests

from sentinelhub import (SHConfig, DataCollection, SentinelHubCatalog, SentinelHubRequest, BBox, bbox_to_dimensions, CRS, MimeType, Geometry)

import sys
sys.path.append('C:\\Users\\****')  # utils.py のあるディレクトリをパスに追加

from utils import plot_image

- `import requests`を最初の行に追加

In [4]:
# 必要なモジュールをインポート
import requests

# Import credentials
from creds import *

def get_keycloak(username: str, password: str) -> str:
    data = {
        "client_id": "cdse-public",
        "username": username,
        "password": password,
        "grant_type": "password",
        }
    try:
        r = requests.post("https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token",
        data=data,
        )
        r.raise_for_status()
    except Exception as e:
        raise Exception(
            f"Keycloak token creation failed. Reponse from the server was: {r.json()}"
            )
    return r.json()["access_token"]
        

keycloak_token = get_keycloak(username, password)